In [ ]:
from llama_index import (
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
    download_loader,
)

import subprocess
import json
import os

openai.api_key = ""
os.environ["OPENAI_API_KEY"] = ""


In [ ]:
# Get YouTube Ids for All In Podcasts
playlist_id = "PLn5MTSAqaf8peDZQ57QkJBzewJU1aUokl"

# Run youtube-dl command with verbose output
command = [
    "youtube-dl",
    "--ignore-errors",
    "--dump-json",
    "--flat-playlist",
    f"--playlist-items",
    f"1-99999",
    f"https://www.youtube.com/playlist?list={playlist_id}",
]
output = subprocess.check_output(command).decode("utf-8")

# Parse the output as JSON
video_info_list = output.strip().split("\n")
video_ids = [json.loads(video_info)["id"] for video_info in video_info_list]

# Convert video IDs to YouTube links
ytlinks = [
    f"https://www.youtube.com/watch?v={video_id}" for video_id in video_ids]


In [ ]:
# No transcripts for these YouTube videos
ytlinks.remove('https://www.youtube.com/watch?v=llJsxamu8f4')
ytlinks.remove('https://www.youtube.com/watch?v=efp-2AHox1U')
ytlinks.remove('https://www.youtube.com/watch?v=B2iNXMiGEms')
ytlinks.remove('https://www.youtube.com/watch?v=eoZG89pDzzY')
ytlinks.remove('https://www.youtube.com/watch?v=jaU1P5-pzLU')
ytlinks.remove('https://www.youtube.com/watch?v=uPwvZZkrJ90')
ytlinks.remove('https://www.youtube.com/watch?v=-XgCOfvSKDQ')
ytlinks.remove('https://www.youtube.com/watch?v=uPwvZZkrJ90')


In [ ]:
# Generate a vector index using YouTube transcripts
YoutubeTranscriptReader = download_loader("YoutubeTranscriptReader")
loader = YoutubeTranscriptReader()
docs = loader.load_data(ytlinks=ytlinks)
index = VectorStoreIndex.from_documents(docs)

# Saving Index for future use
index.storage_context.persist()


In [ ]:
# Querying the index
query_engine = index.as_query_engine()
response = query_engine.query("What is going on with AI startups?")
print(response)
